In [1]:
import pandas as pd
import numpy as np

## EPH survey data

In [242]:
df = pd.read_csv('./../resumen_EPH_hogar.csv')
df = df.loc[df.IV1 != 9]
df['IV10'] = df['IV10'].map({1: 1, 2: 2, 3: 2, 0: 0, 9: 9})
df['II9'] = df['II9'].map({1: 1, 2: 2, 3: 2, 4: 4, 0: 0})
df['II7'] = df['II7'].map({1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 6, 8: 6, 9: 6, 0: 0})
df['IX_TOT'] = df['IX_TOT'].clip(0, 8)


In [3]:
# df.groupby(['AGLOMERADO']).nunique()

In [4]:
# for col in df.iloc[:, 2:].columns:
#     print col
#     print sorted(df[col].unique())

## 2010 ARG Census data

In [262]:
path = './../../Datos_censo/Preguntas/'

VIVIENDA_V01 = pd.read_csv(path + 'VIVIENDA-V01.csv', encoding = 'latin-1')
HOGAR_H05 = pd.read_csv(path + 'HOGAR-H05.csv', encoding = 'latin-1') # Pisos
HOGAR_H06 = pd.read_csv(path + 'HOGAR-H06.csv', encoding = 'latin-1') # Cubierta techos
HOGAR_H07 = pd.read_csv(path + 'HOGAR-H07.csv', encoding = 'latin-1') # Cielorraso
HOGAR_H08 = pd.read_csv(path + 'HOGAR-H08.csv', encoding = 'latin-1') # Agua
HOGAR_H09 = pd.read_csv(path + 'HOGAR-H09.csv', encoding = 'latin-1') # Procedencia agua
HOGAR_H10 = pd.read_csv(path + 'HOGAR-H10.csv', encoding = 'latin-1') # Tiene baño
HOGAR_H11 = pd.read_csv(path + 'HOGAR-H11.csv', encoding = 'latin-1') # Baño con cadena
HOGAR_H12 = pd.read_csv(path + 'HOGAR-H12.csv', encoding = 'latin-1') # Desague baño
HOGAR_H13 = pd.read_csv(path + 'HOGAR-H13.csv', encoding = 'latin-1') # Baño uso exclusivo
HOGAR_H14 = pd.read_csv(path + 'HOGAR-H14.csv', encoding = 'latin-1') # Combustible cocina
HOGAR_PROP = pd.read_csv(path + 'HOGAR-PROP.csv', encoding = 'latin-1') # Regimen tenencia
HOGAR_TOTPERS = pd.read_csv(path + 'HOGAR-TOTPERS.csv', encoding = 'latin-1') # Personas en hogar

df_list = [VIVIENDA_V01, HOGAR_H05, HOGAR_H06, HOGAR_H07, HOGAR_H08, HOGAR_H09, 
          HOGAR_H10, HOGAR_H11, HOGAR_H12, HOGAR_H13, HOGAR_H14, HOGAR_PROP, HOGAR_TOTPERS]

## Harmonise variables (let valid answers match exactly in both datasets)

In [263]:

VIVIENDA_V01['2 Departamento'] = VIVIENDA_V01['4 Departamento']
VIVIENDA_V01['3 Pieza de inquilinato'] = VIVIENDA_V01['5 Pieza en inquilinato']
VIVIENDA_V01[u'4 Pieza en hotel / pensión'] = VIVIENDA_V01[u'6 Pieza en hotel familiar o pensión']
VIVIENDA_V01[u'5 Local no construido para habitación'] = VIVIENDA_V01[u'7 Local no construido para habitación']
VIVIENDA_V01['6 Otros'] = VIVIENDA_V01['2 Rancho'] + VIVIENDA_V01['3 Casilla'] + VIVIENDA_V01[u'8 Vivienda móvil'] + VIVIENDA_V01['9 Persona/s viviendo en la calle']

IV1_censo = VIVIENDA_V01[['radio', '1 Casa', '2 Departamento', '3 Pieza de inquilinato', u'4 Pieza en hotel / pensión',
                         u'5 Local no construido para habitación', '6 Otros', 'TOTAL']]
IV3_censo = HOGAR_H05
IV3_censo.columns = ['radio', '1 Mosaico / baldosa / madera / cerámica / alfombra', 
                    '2 Cemento / ladrillo fijo', '3 Ladrillo suelto / tierra', '4 Otro', 'TOTAL']

IV4_censo = HOGAR_H06
IV4_censo.columns = ['radio', '1 Membrana / cubierta asfáltica', '2 Baldosa / losa sin cubierta', '3 Pizarra / teja', 
                     '4 Chapa de metal sin cubierta', '5 Chapa de fibrocemento / plástico', '6 Chapa de cartón', 
                     '7 Caña / tabla / paja con barro / paja sola', '9 N/S. Depto en propiedad horizontal', 'TOTAL']

IV5_censo = HOGAR_H07
IV6_censo = HOGAR_H08
IV7_censo = HOGAR_H09
IV7_censo['4 Otra fuente'] = HOGAR_H09['4 Pozo'] + HOGAR_H09['5 Transporte por cisterna'] + \
 HOGAR_H09[u'6 Agua de lluvia, río, canal, arroyo o acequia']
IV7_censo = IV7_censo[[u'radio', u'1 Red pública', u'2 Perforación con bomba a motor',
                      u'3 Perforación con bomba manual', u'4 Otra fuente', u'TOTAL']]

IV8_censo = HOGAR_H10
IV10_censo = HOGAR_H11
IV11_censo = HOGAR_H12
II9_censo = HOGAR_H13

II8_censo = HOGAR_H14
II8_censo['2 Gas de tubo / garrafa'] = HOGAR_H14['3 Gas en tubo'] + HOGAR_H14['4 Gas en garrafa']
II8_censo[u'3 Kerosene / leña/ carbón'] = HOGAR_H14[u'6 Leña o carbón']
II8_censo['4 Otro'] = HOGAR_H14[u'2 Gas a granel (zeppelin)'] + HOGAR_H14[u'5 Electricidad'] + HOGAR_H14[u'7 Otro']

II8_censo = II8_censo[['radio', '1 Gas de red', '2 Gas de tubo / garrafa', u'3 Kerosene / leña/ carbón', '4 Otro', 'TOTAL']]

II7_censo = HOGAR_PROP

IX_TOT_censo = HOGAR_TOTPERS

## Intento Max Ent

#### Margins

In [264]:
# From whole country data, we can only have margins. Same for a given district or precint.

tot_viviendas = 12171675.
IV5_censo.iloc[:, 1:].sum()/tot_viviendas

1 Sí     0.723948
2 No     0.276052
TOTAL    1.000000
dtype: float64

In [265]:
IV3_censo.iloc[:, 1:].sum()/tot_viviendas

1 Mosaico / baldosa / madera / cerámica / alfombra    0.765829
2 Cemento / ladrillo fijo                             0.202845
3 Ladrillo suelto / tierra                            0.025842
4 Otro                                                0.005484
TOTAL                                                 1.000000
dtype: float64

In [266]:
IV6_censo.iloc[:, 1:].sum()/tot_viviendas

1 Por cañeria dentro de la vivienda               0.885473
2 Fuera de la vivienda pero dentro del terreno    0.092493
3 Fuera del terreno                               0.022033
TOTAL                                             1.000000
dtype: float64

#### Prior probabilities (matrix entries) from survey info

In [267]:
qjkl = df.groupby(['IV4', 'IV3', 'IV6'])[['CODUSU']].nunique() # how many IDs with each combination
qjkl /= qjkl.sum()

q_ix = pd.MultiIndex.from_product(qjkl.index.levels, names=qjkl.index.names)
qjkl = qjkl.reindex(q_ix, fill_value = 0)

qjkl = qjkl.unstack([2, 1]).sort_index(axis = 1)

info_dfs = [IV4_censo, IV6_censo, IV3_censo]
T = len(info_dfs)

margins_s = []
margins_v = []
shape_ = np.zeros(T, dtype = int)

for t in range(T):
    margin_series = (info_dfs[t].iloc[:, 1:-1].sum()/info_dfs[t].TOTAL.sum())
    margins_s += [margin_series]
    margins_v += [margin_series.values]
    shape_[t] = len(margin_series)
    
a, b, c = tuple([m.values for m in margins_s])
df_ = pd.DataFrame(np.prod(np.ix_(a, b, c)).reshape(qjkl.values.shape), 
                  columns = qjkl.columns, index = qjkl.index)
uncorr_fill_p = df_

uncorr_fill_q = df_ * (qjkl == 0).astype(int)
qjkl = (1 - uncorr_fill_q.sum().sum())*qjkl + uncorr_fill_q

In [268]:
l = np.prod(shape_)
cube = np.arange(l).reshape(shape_)
# cube[:, 2, :].flatten()

In [269]:
## Use the 2D shape of the data frame

In [273]:
from scipy.optimize import minimize
M_0 = qjkl.values#.reshape(8, 3, 4)[0:2]

constraints = []

shape_df = M_0.shape
shape_ #was defined before
l = np.prod(shape_df)
table = np.arange(l).reshape(shape_df)

for dim in range(T):
    cube_tr = cube.transpose(tuple(np.roll(np.arange(T), - (dim)))) 
    marg = margins_v[dim]
    print cube_tr.shape
    print marg
    
    for j in range(shape_[dim]):
        def f(M, cube_tr = cube_tr, marg = marg, j = j):
            return (M.flatten()[cube_tr[j].flatten()]).sum() - marg[j]

        constraints.append(f)
    
cons = [{'type':'eq', 'fun': con} for con in constraints]
cons

def MaxEnt(p, q = M_0.flatten()):
#     p_mat = np.array(p_mat).reshape(q_mat.shape)
    return (p*np.log(p/q)).sum()
    
res = minimize(lambda M: MaxEnt(M), M_0 + 0.001, constraints = cons, options={'maxiter': 1000, 'ftol': 0.01 })

(8, 3, 4)
[ 0.17928839  0.30580031  0.09560977  0.3615159   0.02412938  0.00890009
  0.01495176  0.0098044 ]
(3, 4, 8)
[ 0.88547345  0.09249319  0.02203337]
(4, 8, 3)
[ 0.76582935  0.20284505  0.0258418   0.0054838 ]


/home/miglesia/anaconda2/lib/python2.7/site-packages/IPython/kernel/__main__.py:28: RuntimeWarning: invalid value encountered in log


In [274]:
p = pd.DataFrame(res['x'].reshape(qjkl.shape), columns = qjkl.columns, index = qjkl.index)

In [275]:
p.round(5)

CODUSU                                                                 \
IV6        1                                   2                              
IV3        1        2        3        4        1        2        3        4   
IV4                                                                           
1    0.17850  0.00008  0.00015  0.00011  0.00019  0.00019  0.00003  0.00023   
2    0.13920  0.09629  0.00507  0.00208  0.02264  0.03136  0.00496  0.00155   
3    0.08916  0.00189  0.00021  0.00007  0.00018  0.00022  0.00139  0.00006   
4    0.28368  0.04542  0.00188  0.00030  0.00701  0.01664  0.00519  0.00008   
5    0.01876  0.00304  0.00025  0.00022  0.00000  0.00000  0.00001  0.00000   
6    0.00089  0.00062  0.00040  0.00003  0.00000  0.00000  0.00000  0.00000   
7    0.00481  0.00293  0.00054  0.00003  0.00000  0.00000  0.00000  0.00000   
9    0.01036  0.00005  0.00002  0.00009  0.00000  0.00000  0.00000  0.00000   

                                         
IV6        3                             
IV3        1        2        3        4  
IV4                                      
1    0.00015  0.00006  0.00050  0.00071  
2    0.00043  0.00102  0.00023  0.00012  
3    0.00192  0.00023  0.00012  0.00001  
4    0.00034  0.00025  0.00022  0.00003  
5    0.00029  0.00035  0.00119  0.00003  
6    0.00413  0.00045  0.00242  0.00002  
7    0.00489  0.00076  0.00103  0.00001  
9    0.00023  0.00006  0.00004  0.00000

In [17]:
qjkl.round(5)

CODUSU                                                                 \
IV6        1                                   2                              
IV3        1        2        3        4        1        2        3        4   
IV4                                                                           
1    0.26541  0.01281  0.00081  0.00071  0.00086  0.00112  0.00006  0.00009   
2    0.20764  0.02167  0.00029  0.00043  0.00146  0.00233  0.00013  0.00016   
3    0.06976  0.00202  0.00007  0.00012  0.00007  0.00013  0.00023  0.00005   
4    0.24410  0.05471  0.00074  0.00043  0.00274  0.01023  0.00090  0.00004   
5    0.00986  0.00223  0.00006  0.00018  0.00022  0.00035  0.00018  0.00001   
6    0.00052  0.00049  0.00009  0.00003  0.00003  0.00025  0.00021  0.00001   
7    0.00310  0.00257  0.00018  0.00003  0.00012  0.00043  0.00032  0.00001   
9    0.07036  0.00043  0.00001  0.00040  0.00034  0.00009  0.00002  0.00000   

                                         
IV6        3                             
IV3        1        2        3        4  
IV4                                      
1    0.00004  0.00001  0.00010  0.00002  
2    0.00006  0.00024  0.00001  0.00004  
3    0.00161  0.00043  0.00005  0.00001  
4    0.00031  0.00060  0.00010  0.00004  
5    0.00001  0.00003  0.00003  0.00000  
6    0.00015  0.00003  0.00004  0.00000  
7    0.00025  0.00007  0.00003  0.00000  
9    0.00017  0.00004  0.00001  0.00000

In [276]:
result_p_st = p.stack().stack().reset_index()

print result_p_st.groupby('IV6')['CODUSU'].sum()

print result_p_st.groupby('IV4')['CODUSU'].sum()

print result_p_st.groupby('IV3')['CODUSU'].sum()

IV6
1    0.887161
2    0.091963
3    0.022240
Name: CODUSU, dtype: float64
IV4
1    0.180919
2    0.304957
3    0.095481
4    0.361025
5    0.024144
6    0.008970
7    0.015011
9    0.010857
Name: CODUSU, dtype: float64
IV3
1    0.767781
2    0.201914
3    0.025882
4    0.005787
Name: CODUSU, dtype: float64


In [277]:
margins_v

[array([ 0.17928839,  0.30580031,  0.09560977,  0.3615159 ,  0.02412938,
         0.00890009,  0.01495176,  0.0098044 ]),
 array([ 0.88547345,  0.09249319,  0.02203337]),
 array([ 0.76582935,  0.20284505,  0.0258418 ,  0.0054838 ])]

### Shapefiles aglomerados

In [21]:
import geopandas as gpd
import shapely.geometry as geometry


In [135]:
# gdf = gpd.read_file('./../radios_censo_2010_aglos/radios_censo_2010_aglos.shp')
# gdf = gdf.drop('AREA', axis = 1)
# gdf['geometry'] = [geometry.Point(xy) for xy in zip(gdf.X, gdf.Y)]
# gdf.to_file(driver = 'ESRI Shapefile', filename = './aglos_w_centroid.shp')

# g1 = gpd.GeoDataFrame.from_file("aglos_w_centroid.shp")
# g2 = gpd.GeoDataFrame.from_file("./../radios_w_geometry.shp")

merged = g1[['REDCODE','AGLO_EPH', 'geometry']].merge(g2[['LINK', 'PROVINCIA', 'POBLACION_', 'VIVIENDAS_', 'HOGARES', 'geometry']]
                                             , left_on = ['REDCODE'], right_on = ['LINK'])

In [179]:
correccion_AGLOS = { u'0001': 32, #capital CABA
 u'0002': 13, #cordoba
 u'0003': 4, #rosario
 u'0004': 10, #mendoza
 u'0005': 2, #la plata
 u'0006': 29, #tucuman
 u'0007': 34, #mar del plata
 u'0008': 5, #sta fe
 u'0009': 23,# salta
 u'0010': 27,# san juan
 u'0011': 8,# resistencia
 u'0012': 18, #sgo del estero
 u'0013': 12, #corrientes
 u'0014': 3,# bahia blanca
 u'0015': 17, #neuquen
 u'0016': 6, #parana
 u'0017': 7, #posadas
 u'0018': 19, #jujuy
 u'0019': 15, #formosa
 u'0020': 36, #rio cuarto
 u'0021': 22, #catamarca
 u'0022': 9, #cdro rivadavia
 u'0023': 23, #concordia
 u'0024': 38, #san nicolas
 u'0025': 26, #san luis
 u'0026': 25, #la rioja
 u'0029': 30, #santa rosa
 u'0044': 20, # Rio gallegos
 u'0050': 93, #Viedma (patagones ? )
 u'0086': 31, #Ushuaia rio grande
 u'0135': 91, #Rawson - trelew
 u'1000': 33 }#  GBA
merged['AGLOMERADO'] = merged['AGLO_EPH'].map(correccion_AGLOS)


In [173]:
#Poblacion de aglomerados
# merged.groupby('AGLO_EPH')[['POBLACION_', 'VIVIENDAS_', 'HOGARES']].sum()

In [192]:
VIEDMA = merged.loc[merged.AGLOMERADO == 93][['LINK']].rename(columns = {'LINK': 'radio'})

## Ajuste de los radios censales de un aglomerado

In [228]:
VIEDMA['radio'] = VIEDMA['radio'].astype(int)
IV3_censo['radio'] = IV3_censo['radio'].astype(int)

for df in [IV1_censo, IV3_censo, IV4_censo, IV5_censo, IV6_censo,
           IV7_censo, IV8_censo, IV10_censo, IV11_censo, II9_censo, II8_censo, II7_censo, IX_TOT_censo]:
    
    df.iloc[:, 1:-1] = df.iloc[:, 1:-1].div(df['TOTAL'], axis = 0)

/home/miglesia/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [231]:
merged = VIEDMA.merge(IV1_censo.iloc[:, :-1]
                ).merge(IV3_censo.iloc[:, :-1], on = 'radio'
                       ).merge(IV4_censo.iloc[:, :-1], on = 'radio'
                              ).merge(IV5_censo.iloc[:, :-1], on = 'radio'
                                     ).merge(IV6_censo.iloc[:, :-1], on = 'radio')


In [234]:
merged.loc[2].round(2)

radio                                                 6.200705e+08
1 Casa                                                7.500000e-01
2 Departamento                                        2.400000e-01
3 Pieza de inquilinato                                0.000000e+00
4 Pieza en hotel / pensión                            0.000000e+00
5 Local no construido para habitación                 0.000000e+00
6 Otros                                               1.000000e-02
1 Mosaico / baldosa / madera / cerámica / alfombra    9.300000e-01
2 Cemento / ladrillo fijo                             7.000000e-02
3 Ladrillo suelto / tierra                            0.000000e+00
4 Otro                                                0.000000e+00
1 Membrana / cubierta asfáltica                       3.000000e-02
2 Baldosa / losa sin cubierta                         3.000000e-01
3 Pizarra / teja                                      1.300000e-01
4 Chapa de metal sin cubierta                         5.200000

In [240]:
tuple(range(5 - 1, 0, -1))

(4, 3, 2, 1)

In [278]:
info_dfs = [IV1_censo, IV3_censo, IV4_censo, IV5_censo, IV6_censo]
T = len(info_dfs)

qjkl = df.groupby(['IV1', 'IV3', 'IV4', 'IV5', 'IV6'])[['CODUSU']].nunique() # how many IDs with each combination
qjkl /= qjkl.sum()

q_ix = pd.MultiIndex.from_product(qjkl.index.levels, names=qjkl.index.names)
qjkl = qjkl.reindex(q_ix, fill_value = 0)


qjkl = qjkl.unstack(tuple(range(T - 1, 0, -1))).sort_index(axis = 1)

margins_s = []
margins_v = []
shape_ = np.zeros(T, dtype = int)

for t in range(T):
    margin_series = (info_dfs[t].iloc[:, 1:-1].sum()/info_dfs[t].TOTAL.sum())
    margins_s += [margin_series]
    margins_v += [margin_series.values]
    shape_[t] = len(margin_series)
    
a, b, c, d, e = tuple([m.values for m in margins_s])
df_ = pd.DataFrame(np.prod(np.ix_(a, b, c, d, e)).reshape(qjkl.values.shape), 
                  columns = qjkl.columns, index = qjkl.index)
uncorr_fill_p = df_

uncorr_fill_q = df_ * (qjkl == 0).astype(int)
qjkl = (1 - uncorr_fill_q.sum().sum())*qjkl + uncorr_fill_q

In [279]:
qjkl

CODUSU                                                              \
IV6         1                                                               
IV5         1                                                               
IV4         1                                       2                       
IV3         1         2         3         4         1         2         3   
IV1                                                                         
1    0.139825  0.005215  0.000366  0.000293  0.099717  0.007869  0.000063   
2    0.035374  0.000805  0.000063  0.000167  0.033546  0.000899  0.028428   
3    0.000449  0.000115  0.000031  0.000165  0.000314  0.000031  0.000736   
4    0.000303  0.000073  0.000021  0.000055  0.000251  0.000042  0.000248   
5    0.000021  0.000012  0.000003  0.000045  0.000042  0.000001  0.000200   
6    0.000052  0.000312  0.000074  0.001139  0.000052  0.000010  0.005094   

                                       ...                                   \
IV6                                    ...                  3                 
IV5                                    ...                  2                 
IV4                   3                ...                  6                 
IV3         4         1         2      ...                  3             4   
IV1                                    ...                                    
1    0.000188  0.043202  0.001066      ...       3.135094e-05  1.540872e-05   
2    0.000094  0.003877  0.000052      ...       2.476574e-07  3.795164e-06   
3    0.000077  0.000021  0.000010      ...       6.415536e-09  9.831328e-08   
4    0.000026  0.000010  0.000094      ...       2.158741e-09  3.308108e-08   
5    0.000021  0.000005  0.000076      ...       1.739054e-09  2.664970e-08   
6    0.000532  0.000127  0.001942      ...       4.437840e-08  6.800656e-07   

                                                                           \
IV6                                                                         
IV5                                                                         
IV4             7                                                       9   
IV3             1             2             3             4             1   
IV1                                                                         
1    1.609537e-06  3.834176e-07  1.045031e-05  2.767866e-06  6.593503e-07   
2    3.964284e-07  9.443563e-08  6.526415e-06  6.817245e-07  1.623978e-07   
3    1.026943e-08  2.446344e-09  1.690660e-07  1.765999e-08  4.206895e-09   
4    3.455524e-09  8.231615e-10  5.688842e-08  5.942347e-09  1.415563e-09   
5    2.783726e-09  6.631284e-10  4.582858e-08  4.787079e-09  1.140359e-09   
6    7.103706e-08  1.692217e-08  1.045031e-05  1.221600e-07  2.910049e-08   

                                               
IV6                                            
IV5                                            
IV4                                            
IV3             2             3             4  
IV1                                            
1    1.010405e-05  1.055430e-06  2.514205e-07  
2    2.488624e-06  2.599522e-07  6.192480e-08  
3    6.446751e-08  6.734031e-09  1.604155e-09  
4    2.169244e-08  2.265910e-09  5.397762e-10  
5    1.747515e-08  1.825388e-09  4.348368e-10  
6    4.459432e-07  4.658153e-08  1.109647e-08  

[6 rows x 192 columns]

### Lagrangian approach

In [ ]:
# def L(x, q_mat, margins):
#     shape = np.array(q_mat.shape)
#     len_a = shape.sum()
#     p_mat = np.array(x[:-len_a]).reshape(shape)
#     a = np.array(x[-len_a:])
    
#     if np.random.rand() < 0.0001:
#         print p_mat
#         print a

#     MaxEnt = (p_mat*np.log(p_mat/q_mat))
    
#     a_ix = np.insert(shape.cumsum(), [0], 0)
    
#     lambda_terms = np.zeros(len(shape))
#     for dim in range(len(shape)):
#         lambda_terms[dim] = np.dot(a[a_ix[dim]:a_ix[dim + 1]], 
#                                    margins[dim].values - p_mat.sum(axis = tuple([i for i in range(len(shape)) if i != dim])))

#     return MaxEnt.sum() + lambda_terms.sum()